<a href="https://colab.research.google.com/github/Rajesh16702/AGENT/blob/master/DL_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep learning Assignment - 1 set 3

#To read Kaggle Dataset directly, we need to have kaggle library installed and API keys json uploaded

In [ ]:
!pip install kaggle

Click on ‘Choose Files’ to upload the kaggle.json file

In [ ]:
from google.colab import files
files.upload()

 Move the kaggle.json file into the folder where the API expects to find it

In [ ]:
!mkdir -p ~/.kaggle                                               
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

To search for specific datasets, add the -s tag and then the keyword you’re interested in.

In [ ]:
!kaggle datasets list -s dogs

Copied the "Copy API command" from Dataset tab of Kaggle page to access the dataset

kaggle datasets download -d chetankv/dogs-cats-images